![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings_JSL/Healthcare/24.Improved_Entity_Resolvers_in_SparkNLP_with_sBert.ipynb)

# 24. Improved Entity Resolvers in Spark NLP with sBert

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install -q johnsnowlabs 

In [ ]:
from google.colab import files
print('Please Upload your John Snow Labs License using the button below')
license_keys = files.upload()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
nlp.install()

In [ ]:
from johnsnowlabs import nlp, medical, visual

# Automatically load license data and start a session with all jars user has access to
spark = nlp.start()

<h1>!!! Warning !!!</h1>

**If you get an error related to Java port not found 55, it is probably because that the Colab memory cannot handle the model and the Spark session died. In that case, try on a larger machine or restart the kernel at the top and then come back here and rerun.**

## Utility functions

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='icd10cm_code', hcc=False, aux_label=False):
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    all_distances =[]
    all_codes=[]
    all_cosines = []
    all_k_aux_labels=[]

    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):
            
        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        codes.append(code.result) 
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))
        all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
        
        if hcc:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])
                
        elif aux_label:
            try:
                all_k_aux_labels.append(code.metadata['all_k_aux_labels'].split(':::'))
            except:
                all_k_aux_labels.append([])
        else:
            all_k_aux_labels.append([])

    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'code':codes,'all_codes':all_codes, 
                       'resolutions':resolutions, 'all_k_aux_labels':all_k_aux_labels,'all_distances':all_cosines})
    
    if hcc:

        df['billable'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[0] for i in x])
        df['hcc_status'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[1] for i in x])
        df['hcc_score'] = df['all_k_aux_labels'].apply(lambda x: [i.split('||')[2] for i in x])
    
    elif aux_label:

        df['gt'] = df['all_k_aux_labels'].apply(lambda x: [i.split('|')[0] for i in x])
        df['concept'] = df['all_k_aux_labels'].apply(lambda x: [i.split('|')[1] for i in x])
        df['aux'] = df['all_k_aux_labels'].apply(lambda x: [i.split('|')[2] for i in x])

    df = df.drop(['all_k_aux_labels'], axis=1)
    
    return df

## ICD10CM pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
    .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
    .setInputCols(["ner_chunk"])\
    .setOutputCol("sbert_embeddings")\
    .setCaseSensitive(False)
    
icd10_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
    .setInputCols(["sbert_embeddings"]) \
    .setOutputCol("icd10cm_code")\
    .setDistanceFunction("EUCLIDEAN")

icd_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        icd10_resolver])

icd_lp = nlp.LightPipeline(icd_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented download started this may take some time.
[OK!]


In [ ]:
text = 'bladder cancer'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 73.4 ms, sys: 8.51 ms, total: 81.9 ms
Wall time: 6.05 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]"


In [ ]:
text = 'polydipsia'

%time get_codes (icd_lp, text, vocab='icd10cm_code')

CPU times: user 19.3 ms, sys: 3.26 ms, total: 22.6 ms
Wall time: 1.09 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,polydipsia,0,9,R631,"[R631, F6389, E232, F639, O40, G475, M7989, R632, R061, H538, I442, G479, G478, G471, Q308, I459, R002, R0681, G4754, R068, M353]","[polydipsia, psychogenic polydipsia, primary polydipsia, psychogenic polydipsia (disorder), polyhydramnios, parasomnia, polyalgia, polyphagia, biphasic stridor, oscillopsia, parasystole (disorder), dyssomnia, parasomnia (disorder), hypersomnia (disorder), polyrrhinia, palpitations - irregular, intermittent palpitations, vagal apnea, secondary parasomnia, vagal apnoea, polymyalgia]","[0.0000, 0.0455, 0.0635, 0.0786, 0.1474, 0.1520, 0.1553, 0.1573, 0.1641, 0.1675, 0.1648, 0.1600, 0.1662, 0.1667, 0.1727, 0.1702, 0.1687, 0.1672, 0.1730, 0.1700, 0.1729]"


## ICD10CM-HCC pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
hcc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_hcc_code")\
      .setDistanceFunction("EUCLIDEAN")

hcc_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hcc_resolver])

hcc_lp = nlp.LightPipeline(hcc_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_icd10cm_augmented_billable_hcc download started this may take some time.
[OK!]


In [ ]:
text = 'bladder cancer'

%time get_codes (hcc_lp, text, vocab='icd10cm_hcc_code', hcc=True)

CPU times: user 25.8 ms, sys: 9.94 ms, total: 35.7 ms
Wall time: 1.03 s


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,bladder cancer,0,13,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]","[1, 1, 1, 1, 1]","[1, 0, 0, 0, 1]","[11, 0, 0, 0, 8]"


In [ ]:
text = 'polydipsia'

%time get_codes (hcc_lp, text, vocab='icd10cm_hcc_code', hcc=True)

CPU times: user 17.2 ms, sys: 2.98 ms, total: 20.2 ms
Wall time: 994 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,polydipsia,0,9,R631,"[R631, F6389, E232, F639, O40, G475, M7989, R632, R061, H538, I442, G479, G478, G471, Q308, I459, R002, R0681, G4754, R068, M353]","[polydipsia, psychogenic polydipsia, primary polydipsia, psychogenic polydipsia (disorder), polyhydramnios, parasomnia, polyalgia, polyphagia, biphasic stridor, oscillopsia, parasystole (disorder), dyssomnia, parasomnia (disorder), hypersomnia (disorder), polyrrhinia, palpitations - irregular, intermittent palpitations, vagal apnea, secondary parasomnia, vagal apnoea, polymyalgia]","[0.0000, 0.0455, 0.0635, 0.0786, 0.1474, 0.1520, 0.1553, 0.1573, 0.1641, 0.1675, 0.1648, 0.1600, 0.1662, 0.1667, 0.1727, 0.1702, 0.1687, 0.1672, 0.1730, 0.1700, 0.1729]","[1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1]","[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, nan, 23, nan, 0, 0, 0, 0, 0, 0, 96, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40]"


## RxNorm pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
rxnorm_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        rxnorm_resolver])

rxnorm_lp = nlp.LightPipeline(rxnorm_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_rxnorm_augmented download started this may take some time.
[OK!]


In [ ]:
text = 'metformin 100 mg'

%time get_codes (rxnorm_lp, text, vocab='rxnorm_code')

CPU times: user 17.6 ms, sys: 6.55 ms, total: 24.1 ms
Wall time: 1.09 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,metformin 100 mg,0,15,2200518,"[2200518, 861024, 141916, 334738, 332848, 104494, 199955, 333262, 439563, 429178, 103910, 429408, 450523, 1744000, 401938, 402346, 1726496, 485246, 198758, 316350, 213092]","[metformin hydrochloride 100 MG/ML Extended Release Suspension, metformin hydrochloride 100 MG/ML, fenofibrate 100 MG Oral Capsule, fenofibrate 100 MG, ciprofibrate 100 MG, ciprofibrate 100 MG Oral Tablet, rutin 100 MG Oral Tablet, rutin 100 MG, fendiline 100 MG, fendiline 100 MG Oral Tablet, carbamazepine 100 MG Oral Tablet [Epimaz], perazine 100 MG Oral Tablet, perazine 100 MG, emtricitabine 100 MG, miglustat 100 MG Oral Capsule, miglustat 100 MG, azacitidine 100 MG, azacitidine 100 MG Injection, niacin 100 MG Oral Capsule, niacin 100 MG, dolasetron 100 MG Oral Tablet [Anzemet]]","[0.0658, 0.0658, 0.0684, 0.0684, 0.0697, 0.0697, 0.0765, 0.0765, 0.0775, 0.0775, 0.0780, 0.0804, 0.0804, 0.0818, 0.0811, 0.0811, 0.0812, 0.0812, 0.0811, 0.0811, 0.0836]"


In [ ]:
text = 'Coumadin 5 mg'

%time get_codes (rxnorm_lp, text, vocab='rxnorm_code')

CPU times: user 19.1 ms, sys: 7.57 ms, total: 26.7 ms
Wall time: 975 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Coumadin 5 mg,0,12,855333,"[855333, 432467, 438740, 153692, 352120, 1036890, 104363, 201269, 351399, 668467, 668694, 200795, 153618, 108781, 2475858, 2475857, 905173, 429252, 450946, 1091499, 246155, 439105, 1191685, 206956, 207641]","[warfarin sodium 5 MG [Coumadin], coumarin 5 MG Oral Tablet, coumarin 5 MG, oxybutynin chloride 5 MG Oral Tablet [Urimin], nitisinone 5 MG Oral Capsule [Orfadin], protriptyline hydrochloride 5 MG [Concordin], terazosin 5 MG Oral Tablet [Hytrin], metolazone 5 MG Oral Tablet [Metenix], prednisone 5 MG Oral Tablet [Prednicot], loratadine 5 MG [Claritin], bisacodyl 5 MG [Veracolate], bendroflumethiazide 5 MG Oral Tablet [Urizid], oxybutynin chloride 5 MG Oral Tablet [Contimin], nifedipine 5 MG Oral Capsule [Calanif], vericiguat 5 MG Oral Tablet, vericiguat 5 MG, protriptyline hydrochloride 5 MG [Vivactil], pridinol 5 MG Oral Tablet, pridinol 5 MG, methylphenidate hydrochloride 5 MG [Ritalin], propinox 5 MG Oral Tablet, propinox 5 MG, prednisolone 5 MG [Prednistab], prednisone 5 MG Oral Tablet [Prednicen-M], prochlorperazine 5 MG Rectal Suppository [Compazine]]","[0.0000, 0.0287, 0.0287, 0.0479, 0.0518, 0.0525, 0.0578, 0.0585, 0.0622, 0.0643, 0.0636, 0.0634, 0.0647, 0.0630, 0.0647, 0.0647, 0.0662, 0.0666, 0.0666, 0.0670, 0.0684, 0.0684, 0.0692, 0.0703, 0.0698]"


## NDC pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
ndc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_ndc", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("ndc_code")\
      .setDistanceFunction("EUCLIDEAN")

ndc_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        ndc_resolver])

ndc_lp = nlp.LightPipeline(ndc_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_ndc download started this may take some time.
[OK!]


In [ ]:
text = 'metformin 500 mg'

%time get_codes (ndc_lp, text, vocab='ndc_code')

CPU times: user 18 ms, sys: 4.57 ms, total: 22.5 ms
Wall time: 897 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,metformin 500 mg,0,15,62207-0491,"[62207-0491, 50742-0633, 49483-0622, 62207-0416, 67296-0712, 70010-0063, 75834-0500, 61919-0769, 71800-0008, 70010-0491, 79739-7076, 70882-0124, 42571-0360, 42806-0405, 0069-0136, 49483-0382, 67253-0660, 34645-2105, 59630-0574]","[metformin er 500 mg, metformin 500 mg/1, metformin hcl 500 mg/1, metformin hcl 500 mg/kg, metformin hydrochloride 500 mg/1, metformin hydrochloride 500 mg/500mg, metformin hydrochloride er 500 mg/1, metformin hydrochloride e/r 500 mg/1, metformin hydrochloride 500 mg/5ml, metformin 500 mg/1 tablet, nabumetone 500 mg, metformin hcl 500 mg/1 tablet, metformin hydrochloride oral solution 500 mg/5ml, metformin hydrochloride tablets 500 mg/1, bosulif 500 mg/1, aspirin 500 mg/5001, pyrazinamide 500 mg/1, glycerin 500 mg/ml, fortamet 500 mg/1]","[0.0183, 0.0191, 0.0355, 0.0440, 0.0667, 0.0677, 0.0716, 0.0766, 0.0795, 0.1000, 0.1024, 0.1023, 0.1046, 0.1083, 0.1117, 0.1116, 0.1133, 0.1145, 0.1162]"


In [ ]:
text = 'aspirin 81 mg'

%time get_codes (ndc_lp, text, vocab='ndc_code')

CPU times: user 16.3 ms, sys: 6.4 ms, total: 22.7 ms
Wall time: 859 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,aspirin 81 mg,0,12,41250-0780,"[41250-0780, 72036-0080, 17714-0009, 49483-0349, 73089-0081, 46122-0598, 43063-0862, 69168-0318, 68210-0015, 49035-0467, 51824-0055, 68210-0150, 55910-0274, 67651-0361, 41520-0274, 36800-0467, 69168-0288, 52124-0012, 54257-0274, 70692-0126, 49638-0101, 55910-0467]","[aspirin 81 mg, aspirin 81mg, aspirin 81 mg/1, aspirin 81 mg/811, aspirin 81 mg/81mg, aspirin 81 81 mg/1, aspirin ec 81 mg/1, aspirin 81 mg tablet, assured aspirin 81 mg/1, equate aspirin 81 mg/1, careall aspirin 81 mg/1, valumeds aspirin 81 mg/1, rexall aspirin 81 mg/1, aspirin 81 mg/1 capsule, careone aspirin 81 mg/1, topcare aspirin 81 mg/1, aspirin chewable 81 mg/1, chewable aspirin 81 mg/1, enteric coated aspirin 81 mg/1, aspirin 81mg tablet, aspica (aspirin) 81 mg/1, dg health aspirin 81 mg/1]","[0.0000, 0.0152, 0.0181, 0.0253, 0.0317, 0.0323, 0.0338, 0.0353, 0.0389, 0.0388, 0.0411, 0.0416, 0.0422, 0.0503, 0.0546, 0.0547, 0.0574, 0.0631, 0.0644, 0.0643, 0.0672, 0.0677]"


## CPT pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
cpt_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_procedures_measurements_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("cpt_code")\
      .setDistanceFunction("EUCLIDEAN")

cpt_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        cpt_resolver])

cpt_lp = nlp.LightPipeline(cpt_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_cpt_procedures_measurements_augmented download started this may take some time.
[OK!]


In [ ]:
text = 'heart surgery'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 15.5 ms, sys: 3.43 ms, total: 19 ms
Wall time: 323 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,heart surgery,0,12,33999,"[33999, 33258, 1018145, 0304T, 0167T, 93583, 1006057, 33736, 1013106, 33944, 33310, 33548, 33945, 62319, 33430]","[heart surgery [Unlisted procedure, cardiac surgery], Cardiac surgery procedure [Operative tissue ablation and reconstruction of atria, performed at the time of other cardiac procedure(s), extensive (eg, maze procedure), without cardiopulmonary bypass (List separately in addition to code for primary procedure)], Other Cardiac Surgery Procedures [Other Cardiac Surgery Procedures], Heart procedure [Insertion or removal and replacement of intracardiac ischemia monitoring system including imaging supervision and interpretation when performed and intra-operative interrogation and programming when performed; device only], Heart implantation [Transmyocardial transcatheter closure of ventricular septal defect, with implant; with cardiopulmonary bypass], Operation on cardiac septum [Percutaneous transcatheter septal reduction therapy (eg, alcohol septal ablation) including temporary pacemaker insertion when performed], Surgical Procedures on the Heart and Pericardium [Surgical Procedures on the Heart and Pericardium], Open heart atrial septostomy with cardiopulmonary bypass [Atrial septectomy or septostomy; open heart with cardiopulmonary bypass], Repair Procedures of Structural Heart Defect [Repair Procedures of Structural Heart Defect], Preparation of donor heart for transplantation [Backbench standard preparation of cadaver donor heart allograft prior to transplantation, including dissection of allograft from surrounding soft tissues to prepare aorta, superior vena cava, inferior vena cava, pulmonary artery, and left atrium for im...], Exploratory cardiotomy [Cardiotomy, exploratory (includes removal of foreign body, atrial or ventricular thrombus); without bypass], Surgical ventricular restoration procedure [Surgical ventricular restoration procedure, includes prosthetic patch, when performed (eg, ventricular remodeling, SVR, SAVER, Dor procedures)], Heart transplant [Heart transplant, with or without recipient cardiectomy], Insertion of catheter into heart chamber [Injection(s), including indwelling catheter placement, continuous infusion or intermittent bolus, of diagnostic or therapeutic substance(s) (including anesthetic, antispasmodic, opioid, steroid, other solution), not including neurolytic substances, includ], Replacement of mitral valve with cardiopulmonary bypass [Replacement, mitral valve, with cardiopulmonary bypass]]","[0.0000, 0.1031, 0.1078, 0.1377, 0.1574, 0.1645, 0.1703, 0.1780, 0.1779, 0.1784, 0.1891, 0.1871, 0.1890, 0.1904, 0.1932]"


In [ ]:
text = 'ct abdomen'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 13.3 ms, sys: 0 ns, total: 13.3 ms
Wall time: 311 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,ct abdomen,0,9,74150,"[74150, 35182, 70481, 62284, 74183, 64760, 76497, 61596, 70486, 76700, 31587, 35092, 0157T, 74175, 70490, 74160, 1002900, 71250, 49436]","[CT scan abdomen [Computed tomography, abdomen; without contrast material], RPR CONGENITAL AV FISTULA THORAX & ABDOMEN [Repair, congenital arteriovenous fistula; thorax and abdomen], CT of orbits [Computed tomography, orbit, sella, or posterior fossa or outer, middle, or inner ear; with contrast material(s)], CT of spine [Injection procedure for myelography and/or computed tomography, lumbar], MRI of abdomen [Magnetic resonance (eg, proton) imaging, abdomen; without contrast material(s), followed by with contrast material(s) and further sequences], Incision or removal of nerve to abdomen [Transection or avulsion of; vagus nerve (vagotomy), abdominal], CT scan [Unlisted computed tomography procedure (eg, diagnostic, interventional)], TRANSCOCHLR POST CRNL FOSSA W/WO MOBIL NRV/ART [Transcochlear approach to posterior cranial fossa, jugular foramen or midline skull base, including labyrinthectomy, decompression, with or without mobilization of facial nerve and/or petrous carotid artery], CT scan of face [Computed tomography, maxillofacial area; without contrast material], US abdominal scan [Ultrasound, abdominal, real time with image documentation; complete], Cricoid split procedure [Laryngoplasty, cricoid split, without graft placement], Procedure on artery of abdomen (procedure) [Direct repair of aneurysm, pseudoaneurysm, or excision (partial or total) and graft insertion, with or without patch graft; for ruptured aneurysm, abdominal aorta involving visceral vessels (mesenteric, celiac, renal)], Opening of abdomen [Laparotomy, implantation or replacement of gastric stimulation electrodes, lesser curvature (ie, morbid obesity)], CT of abdominal vascular structures (procedure) [Computed tomographic angiography, abdomen, with contrast material(s), including noncontrast images, if performed, and image postprocessing], CT scan of neck [Computed tomography, soft tissue neck; without contrast material], CT of abdomen with contrast [Computed tomography, abdomen; with contrast material(s)], Anesthesia for Procedures on the Upper Abdomen [Anesthesia for Procedures on the Upper Abdomen], Diagnostic CT scan of chest [Computed tomography, thorax, diagnostic; without contrast material], Creation of exit site for catheter in abdominal cavity [Delayed creation of exit site from embedded subcutaneous segment of intraperitoneal cannula or catheter]]","[0.0756, 0.0985, 0.1099, 0.1268, 0.1351, 0.1422, 0.1526, 0.1571, 0.1570, 0.1534, 0.1619, 0.1549, 0.1559, 0.1614, 0.1638, 0.1645, 0.1591, 0.1694, 0.1672]"


In [ ]:
text = 'Left heart cath'

%time get_codes (cpt_lp, text, vocab='cpt_code')

CPU times: user 12.1 ms, sys: 1.31 ms, total: 13.4 ms
Wall time: 307 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Left heart cath,0,14,93462,"[93462, 33415, 93607, 33988, 33989, 93620, 93514, 36014, 93452, 0281T, 33975, 93531]","[Cardiac catheterization, left heart [Left heart catheterization by transseptal puncture through intact septum or by transapical puncture (List separately in addition to code for primary procedure)], Incision or removal of tissue below left lower heart valve [Resection or incision of subvalvular tissue for discrete subvalvular aortic stenosis], Left ventricular recording (Deprecated) [Left ventricular recording (Deprecated)], Insertion of left heart vent through chest for blood oxygenation rewarming and return [Insertion of left heart vent by thoracic incision (eg, sternotomy, thoracotomy) for ECMO/ECLS], RMVL LEFT HEART VENT BY THORACIC INCIS ECMO/ECLS [Removal of left heart vent by thoracic incision (eg, sternotomy, thoracotomy) for ECMO/ECLS], Left ventricular recording [Comprehensive electrophysiologic evaluation including insertion and repositioning of multiple electrode catheters with induction or attempted induction of arrhythmia; with right atrial pacing and recording, right ventricular pacing and recording, His bund], Left heart catheterization by left ventricular puncture (Deprecated) [Left heart catheterization by left ventricular puncture (Deprecated)], Insertion of catheter into left pulmonary artery [Selective catheter placement, left or right pulmonary artery], Insertion of catheter into left heart including imaging interpretation and supervision and injection [Left heart catheterization including intraprocedural injection(s) for left ventriculography, imaging supervision and interpretation, when performed], Plication of left atrial appendage [Percutaneous transcatheter closure of the left atrial appendage with implant, including fluoroscopy, transseptal puncture, catheter placement(s), left atrial angiography, left atrial appendage angiography, radiological supervision and interpretation], Insertion of external lower heart chamber blood flow assist device [Insertion of ventricular assist device; extracorporeal, single ventricle], Cardiac catheterization, combined right and left heart [Combined right heart catheterization and retrograde left heart catheterization, for congenital cardiac anomalies]]","[0.1060, 0.1104, 0.1272, 0.1367, 0.1424, 0.1433, 0.1611, 0.1639, 0.1683, 0.1678, 0.1731, 0.1687]"


## SNOMED pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
snomed_ct_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings_aux_concepts","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

snomed_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        snomed_ct_resolver])

snomed_lp = nlp.LightPipeline(snomed_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_snomed_findings_aux_concepts download started this may take some time.
Approximate size to download 4.3 GB
[OK!]


In [ ]:
text = 'bladder cancer'

%time get_codes (snomed_lp, text, vocab='snomed_code', aux_label=True)

CPU times: user 3.24 ms, sys: 11.5 ms, total: 14.8 ms
Wall time: 4.73 s


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,bladder cancer,0,13,399326009,"[399326009, 363455001, 425066001, 255108000, 269607003, 154540000, 425231005, 255110003, 139850000, 162582000, 315269000, 393562002, 255109008, 255111004, 92546004, 254932004]","[bladder cancer, bladder cancer, invasive bladder cancer, carcinoma of bladder, carcinoma of bladder, carcinoma of bladder, superficial bladder cancer, adenocarcinoma of bladder, suspected bladder cancer, suspected bladder cancer, suspected bladder cancer, transitional cell carcinoma of bladder, transitional cell carcinoma of bladder, squamous cell carcinoma of bladder, cancer in situ of urinary bladder, tumor of bladder neck]","[0.0000, 0.0000, 0.0539, 0.0666, 0.0666, 0.0666, 0.0809, 0.0881, 0.0904, 0.0904, 0.0904, 0.0880, 0.0880, 0.0913, 0.0978, 0.1080]","[Malignant tumor of urinary bladder, NGB - New growth of bladder, Carcinoma of urinary bladder, invasive, Carcinoma of bladder, Bladder Ca, Carcinoma bladder, Carcinoma of urinary bladder, superficial, Adenocarcinoma of bladder, Suspected bladder cancer, Suspected bladder cancer, Suspected bladder cancer, Transitional cell carcinoma of bladder, Transitional cell carcinoma of bladder, Squamous cell carcinoma of bladder, Cancer in situ of urinary bladder, Tumor of bladder neck]","[Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Observation, Observation, Observation, Condition, Condition, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Context-dependent, Context-dependent, Context-dependent, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]"


In [ ]:
text = 'schizophrenia'

%time get_codes (snomed_lp, text, vocab='snomed_code', aux_label=True)

CPU times: user 7.81 ms, sys: 3.12 ms, total: 10.9 ms
Wall time: 2.45 s


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,schizophrenia,0,12,58214004,"[58214004, 83746006, 191567000, 191529003, 16990005, 35252006, 137730004, 160328009, 64905009, 161468000, 138739000, 191526005, 12939007, 76834001, 154868009, 76839006, 191542003]","[schizophrenia, chronic schizophrenia, schizoaffective schizophrenia, subchronic schizophrenia, subchronic schizophrenia, disorganized schizophrenia, fh: schizophrenia, fh: schizophrenia, paraphrenic schizophrenia, h/o: schizophrenia, h/o: schizophrenia, schizophrenic disorders, chronic disorganised schizophrenia, retired procedure, catatonic schizophrenia, catatonic schizophrenia, catatonic schizophrenia]","[0.0000, 0.0387, 0.0481, 0.0498, 0.0498, 0.0532, 0.0597, 0.0597, 0.0663, 0.0653, 0.0653, 0.0662, 0.0683, 0.0694, 0.0694, 0.0694, 0.0694]","[Schizophrenia, Chronic schizophrenia, Schizoaffective schizophrenia, Subchronic schizophrenia, Subchronic schizophrenia, Disorganized schizophrenia, FH: Schizophrenia, FH: Schizophrenia, Paranoid schizophrenia, H/O: schizophrenia, H/O: schizophrenia, Schizophrenic disorders, Chronic disorganized schizophrenia, Retired procedure, Catatonic schizophrenia, Catatonic schizophrenia, Catatonic schizophrenia]","[Condition, Condition, Condition, Condition, Condition, Condition, Observation, Observation, Condition, Observation, Observation, Condition, Condition, Procedure, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Context-dependent, Context-dependent, Clinical Finding, Context-dependent, Context-dependent, Clinical Finding, Clinical Finding, Procedure, Clinical Finding, Clinical Finding, Clinical Finding]"


## LOINC Pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
loinc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_loinc_augmented", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("loinc_code")\
      .setDistanceFunction("EUCLIDEAN")

loinc_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        loinc_resolver])

loinc_lp = nlp.LightPipeline(loinc_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_loinc_augmented download started this may take some time.
[OK!]


In [ ]:
text = 'FLT3 gene mutation analysis'

%time get_codes (loinc_lp, text, vocab='loinc_code')

CPU times: user 65.5 ms, sys: 17.2 ms, total: 82.7 ms
Wall time: 8.87 s


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,FLT3 gene mutation analysis,0,26,LP228450-5,"[LP228450-5, LP310331-6, LP228432-3, LP228446-3, LP228445-5, LP229390-2, LP228738-3, LP229796-0, LP229155-9, LP228431-5, LP229833-1, LP228570-0, LP229609-5, LP227517-2, 92843-2, LP229667-3, LP228546-0, LP228828-2, LP229608-7, LP228488-5, LP228434-9, LP228430-7, LP228825-8, LP228479-4]","[flt3 gene targeted mutation analysis, flt3 gene p.asp835 mutations, fgfr3 gene targeted mutation analysis, flna gene targeted mutation analysis, flcn gene targeted mutation analysis, pou3f4 gene targeted mutation analysis, itgb3 gene targeted mutation analysis, tgfb3 gene targeted mutation analysis, opa3 gene targeted mutation analysis, fgfr3 gene mutations tested for, traf3 gene targeted mutation analysis, gpc3 gene targeted mutation analysis, sh3tc2 gene targeted mutation analysis, abca3 gene targeted mutation analysis, flt3 p.d835 mutations, spg3a gene targeted mutation analysis, gjb3 gene targeted mutation analysis, lamb3 gene targeted mutation analysis, sh3bp2 gene targeted mutation analysis, fxn gene targeted mutation analysis, fh gene targeted mutation analysis, fgfr2 gene+fgfr3 gene targeted mutation analysis, lama3 gene targeted mutation analysis, fshd gene targeted mutation analysis]","[0.0354, 0.1032, 0.1021, 0.1111, 0.1135, 0.1258, 0.1249, 0.1285, 0.1255, 0.1316, 0.1287, 0.1328, 0.1346, 0.1315, 0.1358, 0.1351, 0.1352, 0.1339, 0.1390, 0.1347, 0.1318, 0.1387, 0.1362, 0.1360]"


In [ ]:
text = 'Hematocrit'

%time get_codes (loinc_lp, text, vocab='loinc_code')

CPU times: user 16.4 ms, sys: 7.67 ms, total: 24.1 ms
Wall time: 984 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Hematocrit,0,9,LP15101-6,"[LP15101-6, LP308151-2, 32354-3, 20570-8, 11153-4, LP74090-9, 13508-7, %{of'HGB}, 42908-4, LP29006-1, 11559-2, 71832-0, LP16437-3, LP14449-0, 41654-5, LP35956-9, 41986-1, LP16428-2, 48703-3, 71828-8, 55103-6, 8478-0, 41655-2, LP30931-7]","[hematocrit, hematocrit/hemoglobin, hematocrit [volume fraction] of arterial blood, hematocrit [volume fraction] of blood, hematocrit [volume fraction] of body fluid, hematocrit test status, microhematocrit, percent hemoglobin, hematocrit [volume fraction] of capillary blood, blood volume, fractional hemoglobin, hematocrit [pure volume fraction] of arterial blood, hemoglobin i, hemoglobin, hematocrit [volume fraction] of venous blood, hematocrit test status/results, hematocrit test status/results, hemoglobin f, hematocrit [volume fraction] of blood by estimated, hematocrit [pure volume fraction] of body fluid, blood output, mean blood pressure, hematocrit [volume fraction] of mixed venous blood, hemoglobin m]","[0.0000, 0.0506, 0.0590, 0.0625, 0.0675, 0.0740, 0.0737, 0.0996, 0.1035, 0.1010, 0.1060, 0.1075, 0.1082, 0.1122, 0.1139, 0.1162, 0.1162, 0.1166, 0.1196, 0.1194, 0.1167, 0.1193, 0.1245, 0.1245]"


In [ ]:
text = 'vomiting'

%time get_codes (loinc_lp, text, vocab='loinc_code')

CPU times: user 16.6 ms, sys: 5.66 ms, total: 22.2 ms
Wall time: 978 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,vomiting,0,7,LA15099-7,"[LA15099-7, 10693-0, LP7717-4, 81224-8, LA15098-9, LA15902-2, 3609519, LP412237-2, LA25131-6, 81660-3, LA17127-4, 94069-2, 42848-2, 9303492, 6059284, 6074534, 67232-9, 3709730, LA6150-2, 251609, 2421664, LA30521-1, 77542-9, LA16987-2, 87566-6]","[vomiting, vomitus, vomitus, vomiting [ccc], nausea, nausea or vomiting, braughing, emesis count, rumbling, nausea [presence], choking, emesis occurred, nausea [ccc], thorburn, torryburn, goulburn, nausea freq, wooburn, agitation, embarrass, bobbing, groaning respiration, fainting, fainting, vomitoxin]","[0.0000, 0.0175, 0.0175, 0.0589, 0.0755, 0.0846, 0.0900, 0.0949, 0.0985, 0.0980, 0.1009, 0.1012, 0.1034, 0.1051, 0.1085, 0.1102, 0.1082, 0.1115, 0.1095, 0.1121, 0.1136, 0.1108, 0.1105, 0.1105, 0.1182]"


## HCPCS Pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
hcpcs_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_hcpcs", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("hcpcs_code")\
      .setDistanceFunction("EUCLIDEAN")

hcpcs_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hcpcs_resolver])

hcpcs_lp = nlp.LightPipeline(hcpcs_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_hcpcs download started this may take some time.
[OK!]


In [ ]:
text = "Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, unilateral, any size, any type"

%time get_codes (hcpcs_lp, text, vocab='hcpcs_code')

CPU times: user 12.1 ms, sys: 3.55 ms, total: 15.7 ms
Wall time: 133 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,"Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, unilateral, any size, any type",0,104,L8001,"[L8001, L8002, L8000, L8033, L8032, S2067, L8020, S8080, L8010, G9131, G9071, S2066, G9075, K1005, G9073, L8031, G9072, L8015, E0602, L8600, S2068, E0604, G9074, L8030, G9114]","[Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, unilateral, any size, any type, Breast prosthesis, mastectomy bra, with integrated breast prosthesis form, bilateral, any size, any type, Breast prosthesis, mastectomy bra, without integrated breast prosthesis form, any size, any type, Nipple prosthesis, custom fabricated, reusable, any material, any type, each, Nipple prosthesis, prefabricated, reusable, any type, each, Breast reconstruction of a single breast with ""stacked"" deep inferior epigastric perforator (diep) flap(s) and/or gluteal artery perforator (gap) flap(s), including harvesting of the flap(s), microvascular transfer, closure of donor site(s) and shaping the flap into a breast, unilateral, Breast prosthesis, mastectomy form, Scintimammography (radioimmunoscintigraphy of the breast), unilateral, including supply of radiopharmaceutical, Breast prosthesis, mastectomy sleeve, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; extent of disease unknown, staging in progress, or not listed (for use in a medicare-approved demonstration project), Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage i or stage iia-iib; or t3, n1, m0; and er and/or pr positive; with no evidence of disease progression, recurrence, or metastases (for use in a medicare-approved demonstration project), Breast reconstruction with gluteal artery perforator (gap) flap, including harvesting of the flap, microvascular transfer, closure of donor site and shaping the flap into a breast, unilateral, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; m1 at diagnosis, metastatic, locally recurrent, or progressive (for use in a medicare-approved demonstration project), Disposable collection and storage bag for breast milk, any size, any type, each, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage iiia-iiib; and not t3, n1, m0; and er and/or pr positive; with no evidence of disease progression, recurrence, or metastases (for use in a medicare-approved demonstration project), Breast prosthesis, silicone or equal, with integral adhesive, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage i, or stage iia-iib; or t3, n1, m0; and er and pr negative; with no evidence of disease progression, recurrence, or metastases (for use in a medicare-approved demonstration project), External breast prosthesis garment, with mastectomy form, post mastectomy, Breast pump, manual, any type, Implantable breast prosthesis, silicone or equal, Breast reconstruction with deep inferior epigastric perforator (diep) flap or superficial inferior epigastric artery (siea) flap, including harvesting of the flap, microvascular transfer, closure of donor site and shaping the flap into a breast, unilateral, Breast pump, hospital grade, electric (ac and / or dc), any type, Oncology; disease status; invasive female breast cancer (does not include ductal carcinoma in situ); adenocarcinoma as predominant cell type; stage iiia-iiib; and not t3, n1, m0; and er and pr negative; with no evidence of disease progression, recurrence, or metastases (for use in a medicare-approved demonstration project), Breast prosthesis, silicone or equal, without integral ad

In [ ]:
text= "Dietary consultation for carbohydrate counting for type I diabetes."
%time get_codes (hcpcs_lp, text, vocab='hcpcs_code')

CPU times: user 16.9 ms, sys: 4.11 ms, total: 21 ms
Wall time: 106 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Dietary consultation for carbohydrate counting for type I diabetes.,0,66,S9465,"[S9465, S9434, S9460, E0787, G8777, S9435, B4157, B5200, B4162, G8725, G8485, B4149, G0343, A4236, S9470, B5100, A4234, B4155, A4235, G0246, G9147, G4017, B5000, VM, G0342]","[Diabetic management program, dietitian visit, Modified solid food supplements for inborn errors of metabolism, Diabetic management program, nurse visit, External ambulatory infusion pump, insulin, dosage rate adjustment using therapeutic continuous glucose sensing, Diabetes screening test performed, Medical foods for inborn errors of metabolism, Enteral formula, nutritionally complete, for special metabolic needs for inherited disease of metabolism, includes proteins, fats, carbohydrates, vitamins and minerals, may include fiber, administered through an enteral feeding tube, 100 calories = 1 unit, Parenteral nutrition solution compounded amino acid and carbohydrates with electrolytes, trace elements, and vitamins, including preparation, any strength, stress-branch chain amino acids-freamine-hbc-premix, Enteral formula, for pediatrics, special metabolic needs for inherited disease of metabolism, includes proteins, fats, carbohydrates, vitamins and minerals, may include fiber, administered through an enteral feeding tube, 100 calories = 1 unit, Fasting lipid profile performed (triglycerides, ldl-c, hdl-c and total cholesterol), I intend to report the diabetes mellitus (dm) measures group, Enteral formula, manufactured blenderized natural foods with intact nutrients, includes proteins, fats, carbohydrates, vitamins and minerals, may include fiber, administered through an enteral feeding tube, 100 calories = 1 unit, Laparotomy for islet cell transplant, includes portal vein catheterization and infusion, Replacement battery, silver oxide, for use with medically necessary home blood glucose monitor owned by patient, each, Nutritional counseling, dietitian visit, Parenteral nutrition solution compounded amino acid and carbohydrates with electrolytes, trace elements, and vitamins, including preparation, any strength, hepatic, hepatamine-premix, Replacement battery, alkaline, j cell, for use with medically necessary home blood glucose monitor owned by patient, each, Enteral formula, nutritionally incomplete/modular nutrients, includes specific nutrients, carbohydrates (e.g., glucose polymers), proteins/amino acids (e.g., glutamine, arginine), fat (e.g., medium chain triglycerides) or combination, administered through an enteral feeding tube, 100 calories = 1 unit, Replacement battery, lithium, for use with medically necessary home blood glucose monitor owned by patient, each, Follow-up physician evaluation and management of a diabetic patient with diabetic sensory neuropathy resulting in a loss of protective sensation (lops) to include at least the following: (1) a patient history, (2) a physical examination that includes: (a) visual inspection of the forefoot, hindfoot and toe web spaces, (b) evaluation of protective sensation, (c) evaluation of foot structure and biomechanics, (d) evaluation of vascular status and skin integrity, and (e) evaluation and recommendation of footwear, and (3) patient education, Outpatient intravenous insulin treatment (oivit) either pulsatile or continuous, by any means, guided by the results of measurements for: respiratory quotient; and/or, urine urea nitrogen (uun); and/or, arterial, venous or capillary glucose; and/or potassium concentration, Nutrition/dietician mips specialty set, Parenteral nutrition solution compounded amino acid and carbohydrates with electrolytes, trace elements, and vitamins, including preparation, any strength, renal-aminosyn-rf, nephramine, renamine-premix, Medicare diabetes prevention program (mdpp) virtual make-up session, Laparoscopy for islet cell transplant, includes portal vein catheterization and infusion]","[0.2426, 0.3054, 0.3147, 0.3332, 0.3201, 0.3398, 0.

In [ ]:
text= "Psychiatric consultation for alcohol withdrawal and dependance."
%time get_codes (hcpcs_lp, text, vocab='hcpcs_code')

CPU times: user 15.1 ms, sys: 4.75 ms, total: 19.9 ms
Wall time: 109 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Psychiatric consultation for alcohol withdrawal and dependance.,0,62,H0010,"[H0010, H0011, H0007, H0012, H0013, H0008, T1006, G9621, H0009, G2196, S9485, H2036, G2203, H0005, G8959, H2034, G2200, HG, H0014, H0006, G2121, H2018, T1007, G9717, H0022]","[Alcohol and/or drug services; sub-acute detoxification (residential addiction program inpatient), Alcohol and/or drug services; acute detoxification (residential addiction program inpatient), Alcohol and/or drug services; crisis intervention (outpatient), Alcohol and/or drug services; sub-acute detoxification (residential addiction program outpatient), Alcohol and/or drug services; acute detoxification (residential addiction program outpatient), Alcohol and/or drug services; sub-acute detoxification (hospital inpatient), Alcohol and/or substance abuse services, family/couple counseling, Patient identified as an unhealthy alcohol user when screened for unhealthy alcohol use using a systematic screening method and received brief counseling, Alcohol and/or drug services; acute detoxification (hospital inpatient), Patient identified as an unhealthy alcohol user when screened for unhealthy alcohol use using a systematic screening method, Crisis intervention mental health services, per diem, Alcohol and/or other drug treatment program, per diem, Documentation of medical reason(s) for not providing brief counseling if identified as an unhealthy alcohol user (e.g., limited life expectancy, other medical reasons), Alcohol and/or drug services; group counseling by a clinician, Clinician treating major depressive disorder communicates to clinician treating comorbid condition, Alcohol and/or drug abuse halfway house services, per diem, Patient identified as an unhealthy alcohol user received brief counseling, Opioid addiction treatment program, Alcohol and/or drug services; ambulatory detoxification, Alcohol and/or drug services; case management, Depression, anxiety, apathy, and psychosis assessed, Psychosocial rehabilitation services, per diem, Alcohol and/or substance abuse services, treatment plan development and/or modification, Documentation stating the patient has had a diagnosis of depression or has had a diagnosis of bipolar disorder, Alcohol and/or drug intervention service (planned facilitation)]","[0.1938, 0.2086, 0.2104, 0.2134, 0.2177, 0.2423, 0.2430, 0.2509, 0.2554, 0.2594, 0.2716, 0.2658, 0.2729, 0.2695, 0.2716, 0.2755, 0.2708, 0.2776, 0.2933, 0.2799, 0.2904, 0.3020, 0.2915, 0.2835, 0.2985]"


## UMLS Pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
umls_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_umls_major_concepts", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

umls_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        umls_resolver])

umls_lp = nlp.LightPipeline(umls_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_umls_major_concepts download started this may take some time.
[OK!]


In [ ]:
# medical device
text = 'X-Ray'

%time get_codes (umls_lp, text, vocab='umls_code')

CPU times: user 12.6 ms, sys: 5.24 ms, total: 17.8 ms
Wall time: 499 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,X-Ray,0,4,C0043303,"[C0043303, C0184486, C0182605, C0183263, C0183264, C3880096, C0868115, C0221564, C0000766, C0184480, C3880545]","[x-ray film, x-ray tubes, x-ray machine, x-ray shield, x-ray aprons, x-ray system tube (physical object), exposure x ray, specific x-ray findings, x-ray abnormal, x-ray system generator (physical object), x-ray system cover (physical object)]","[0.0416, 0.0662, 0.0712, 0.0906, 0.1029, 0.1180, 0.1201, 0.1210, 0.1165, 0.1286, 0.1314]"


In [ ]:
# Injuries & poisoning
text = 'out-of-date food poisoning'

%time get_codes (umls_lp, text, vocab='umls_code')

CPU times: user 15.2 ms, sys: 4.79 ms, total: 20 ms
Wall time: 511 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,out-of-date food poisoning,0,25,C0016479,"[C0016479, C0349783, C0411266, C0161721, C0417672, C0567797, C0411267, C1281398, C0178496, C4743928, C0343360, C3665356, C4724202, C1820738, C0565937, C4743931]","[food poisoning (disorder), infective food poisoning (disorder), chemical food poisoning (disorder), toxic effect of noxious substance eaten as food (disorder), accident due to lack of food, pectin poisoning of undetermined intent (disorder), heavy metal food poisoning (disorder), inanition due to lack of food (disorder), bacterial food poisoning (disorder), grade 2 feeding intolerance, viral food poisoning (disorder), history of food intolerance, feeding intolerance, feeding intolerance, food burn of skin (disorder), grade 5 feeding intolerance, adverse event]","[0.1007, 0.1104, 0.1117, 0.1331, 0.1431, 0.1496, 0.1519, 0.1546, 0.1548, 0.1585, 0.1625, 0.1618, 0.1624, 0.1624, 0.1645, 0.1660]"


In [ ]:
# clinical findings
text = 'type two diabetes mellitus'

%time get_codes (umls_lp, text, vocab='umls_code')

CPU times: user 18.8 ms, sys: 2.23 ms, total: 21 ms
Wall time: 505 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,type two diabetes mellitus,0,25,C4014362,"[C4014362, C3532488, C1320657, C1313937, C0455488, C2733146, C4017629, C3532489, C3275844, C0241863, C1261139, C0421248, C5195213, C1317301, C3278636, C2675471, C0235397, C4316344, C3872445]","[type 2 diabetes mellitus (t2d), history of diabetes mellitus type 2 (situation), diabete type, fh: diabetes mellitus, pre-existing diabetes mellitus, type 2 diabetes mellitus uncontrolled, diabetes mellitus, type ii, digenic, history of diabetes mellitus type i, diabetes mellitus, type ii, susceptibility to, diabetic, diabetic relative, insulin diabetic, increased risk of type 2 diabetes, diabetes status, neonatal insulin-dependent diabetes mellitus, microvascular complications of diabetes, susceptibility to, 2, diabetes mellitus precipitated, disorder of vision due to secondary diabetes mellitus (disorder), suspected glaucoma due to type 2 diabetes mellitus (situation)]","[0.0438, 0.0568, 0.0618, 0.0697, 0.0780, 0.0891, 0.0910, 0.0883, 0.0940, 0.0939, 0.1031, 0.1062, 0.1087, 0.1104, 0.1116, 0.1201, 0.1138, 0.1198, 0.1243]"


## MeSH pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
mesh_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_mesh","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("mesh_code")\
      .setDistanceFunction("EUCLIDEAN")

mesh_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        mesh_resolver])

mesh_lp = nlp.LightPipeline(mesh_pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_mesh download started this may take some time.
[OK!]


In [ ]:
text = 'chest pain '

%time get_codes (mesh_lp, text, vocab='mesh_code')

CPU times: user 21.8 ms, sys: 4.53 ms, total: 26.3 ms
Wall time: 660 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,chest pain,0,10,D002637,"[D002637, D059350, D019547, D020069, D015746, D000072716, D005157, D059265, D001416, D048949, D021501, D058447, D010146, D059390, C563475, D059352, D053591, D059787, D009209, D010148, D058748, D020918, D017699, D059388, D005409]","[Chest Pain, Chronic Pain, Neck Pain, Shoulder Pain, Abdominal Pain, Cancer Pain, Facial Pain, Visceral Pain, Back Pain, Labor Pain, Flank Pain, Eye Pain, Pain, Breakthrough Pain, Paroxysmal Extreme Pain Disorder, Musculoskeletal Pain, Pain, Referred, Acute Pain, Myofascial Pain Syndromes, Pain, Intractable, Pain Perception, Complex Regional Pain Syndromes, Pelvic Pain, Pelvic Girdle Pain, Flail Chest]","[0.0000, 0.0577, 0.0587, 0.0601, 0.0658, 0.0704, 0.0712, 0.0741, 0.0766, 0.0778, 0.0794, 0.0827, 0.0845, 0.0914, 0.0941, 0.0935, 0.0957, 0.1023, 0.1078, 0.1060, 0.1109, 0.1166, 0.1187, 0.1189, 0.1199]"


In [ ]:
text = 'pericardectomy'

%time get_codes (mesh_lp, text, vocab='mesh_code')

CPU times: user 15.4 ms, sys: 5.06 ms, total: 20.5 ms
Wall time: 660 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,pericardectomy,0,13,D010492,"[D010492, D011670, D018700, D014068, D000541, D005927, D020884, D064727, D010611, D011672, D059526, D001062, D002431, D011013, D013156, D032801, D001047, D020519, D000074885, D038421, D010180, D007825, D000074406, D017128, D056346]","[Pericardiectomy, Pulpectomy, Pleurodesis, Tonsillectomy, Alveolectomy, Glossectomy, Colpotomy, Posterior Capsulotomy, Pharyngectomy, Pulpotomy, Foraminotomy, Appendectomy, Cecostomy, Pneumonectomy, Splenectomy, Iridectomy, Apicoectomy, Pericardiocentesis, Amniotomy, Hemispherectomy, Pancreatectomy, Laryngectomy, Mastoidectomy, Embolectomy, Sternotomy]","[0.0732, 0.1287, 0.1499, 0.1540, 0.1573, 0.1582, 0.1645, 0.1622, 0.1660, 0.1676, 0.1729, 0.1697, 0.1744, 0.1774, 0.1753, 0.1757, 0.1783, 0.1812, 0.1803, 0.1726, 0.1799, 0.1813, 0.1852, 0.1909, 0.1867]"


## HPO Pipeline

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
hpo_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_HPO", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

hpo_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        hpo_resolver])

hpo_lp = nlp.LightPipeline(hpo_pipelineModel)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
sbiobertresolve_HPO download started this may take some time.
[OK!]


In [ ]:
text = 'bladder cancer'

%time get_codes (hpo_lp, text, vocab='umls_code')

CPU times: user 17.3 ms, sys: 6.68 ms, total: 23.9 ms
Wall time: 136 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,bladder cancer,0,13,HP:0009725,"[HP:0009725, HP:0002862, HP:0006740, HP:0030409, HP:0100517, HP:0000015, HP:0100516, HP:0009726, HP:0004375, HP:0004321, HP:0032465, HP:0010786, HP:0010474, HP:0010784, HP:0006762, HP:0005584, HP:0025489, HP:0010477]","[bladder cancer, bladder carcinoma, transitional cell carcinoma of the bladder, renal urothelial carcinoma, neoplasia of the urethra, bladder diverticulum, ureter, cancer of, kidney cancer, nervous system cancer, bladder fistula, bladder trabeculation, urinary tract neoplasia, bladder stones, uterine cancer, renal pelvic carcinoma, renal carcinoma, bladder duplication, aplasia of the bladder]","[0.0000, 0.0503, 0.0905, 0.1612, 0.1729, 0.1738, 0.1839, 0.1886, 0.1924, 0.1892, 0.1930, 0.1903, 0.1904, 0.1938, 0.1987, 0.2013, 0.2006, 0.2039]"


In [ ]:
text = 'bipolar disorder'

%time get_codes (hpo_lp, text, vocab='umls_code')

CPU times: user 12.6 ms, sys: 3.48 ms, total: 16 ms
Wall time: 132 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,bipolar disorder,0,15,HP:0007302,"[HP:0007302, HP:0100753, HP:0012076, HP:0000708, HP:0033516, HP:0012077, HP:0000709, HP:0012075, HP:0000716, HP:0000722, HP:0002134, HP:0011511, HP:0000725, HP:0025102, HP:0002072, HP:0011223, HP:0033512, HP:0100022, HP:0001345, HP:0002269]","[bipolar disorder, schizophrenia, borderline personality disorder, psychiatric disorders, benzodiazepine dependence, histrionic personality disorder, psychosis, personality disorder, depressive disorder, obsessive compulsive disorder, basal ganglia disease, macular schisis, psychotic episodes, basal ganglia dysgenesis, choreatic disease, metopic depression, stimulant dependence, movement disorder, psychotic mentation, migrational brain disorder]","[0.0000, 0.1488, 0.1486, 0.1577, 0.1740, 0.1731, 0.1784, 0.1764, 0.1786, 0.1830, 0.1925, 0.2145, 0.2192, 0.2182, 0.2376, 0.2325, 0.2325, 0.2330, 0.2443, 0.2393]"


In [ ]:
text = 'schizophrenia '

%time get_codes (hpo_lp, text, vocab='umls_code')

CPU times: user 15.9 ms, sys: 3.29 ms, total: 19.2 ms
Wall time: 132 ms


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,schizophrenia,0,13,HP:0100753,"[HP:0100753, HP:0000709, HP:0007302, HP:0000708, HP:0000725, HP:0011511, HP:0012077, HP:0001345, HP:0030325, HP:0012075, HP:0000722, HP:0033512, HP:0012076, HP:0100923, HP:0002072, HP:0033715, HP:0010891, HP:0011626, HP:0100656, HP:0033516, HP:0008404, HP:0002269]","[schizophrenia, psychosis, bipolar disorder, psychiatric disorders, psychotic episodes, macular schisis, histrionic personality disorder, psychotic mentation, cervicomedullary schisis, personality disorder, obsessive compulsive disorder, stimulant dependence, borderline personality disorder, clavicular sclerosis, choreatic disease, mesial temporal sclerosis, scheuermann disease, scimitar syndrome, thoracoabdominal schisis, benzodiazepine dependence, onychodystrophy, migrational brain disorder]","[0.0000, 0.1172, 0.1488, 0.1599, 0.1877, 0.1898, 0.1923, 0.2033, 0.2163, 0.2113, 0.2166, 0.2257, 0.2202, 0.2345, 0.2431, 0.2389, 0.2497, 0.2438, 0.2578, 0.2483, 0.2604, 0.2455]"


## All the resolvers in the same pipeline 
### (just to show how it is done..  will not be used in this notebook)

In [ ]:

documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)
    
snomed_ct_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_snomed_findings","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("snomed_code")\
      .setDistanceFunction("EUCLIDEAN")

icd10_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")

rxnorm_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_rxnorm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("rxnorm_code")\
      .setDistanceFunction("EUCLIDEAN")

cpt_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_cpt_procedures_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("cpt_code")\
      .setDistanceFunction("EUCLIDEAN")

hcc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented_billable_hcc","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_hcc_code")\
      .setDistanceFunction("EUCLIDEAN")

loinc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_loinc_augmented", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("loinc_code")\
      .setDistanceFunction("EUCLIDEAN")

hcpcs_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_hcpcs", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("hcpcs_code")\
      .setDistanceFunction("EUCLIDEAN")

umls_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_umls_major_concepts", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

hpo_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_HPO", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("umls_code")\
      .setDistanceFunction("EUCLIDEAN")

mesh_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_mesh","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("mesh_code")\
      .setDistanceFunction("EUCLIDEAN")

ndc_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_ndc", "en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("ndc_code")\
      .setDistanceFunction("EUCLIDEAN")


resolver_pipelineModel = nlp.PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        icd10_resolver,
        hcc_resolver,
        rxnorm_resolver,
        ndc_resolver,
        cpt_resolver,
        snomed_ct_resolver,
        loinc_resolver,
        hcpcs_resolver,
        umls_resolver,
        mesh_resolver,
        hpo_resolver])

resolver_lp = nlp.LightPipeline(resolver_pipelineModel)


## Getting some predictions from resolvers

In [ ]:
icd_chunks = ['obesity',
'a respiratory tract infection',
'basal cell carcinoma of skin',
'acute maxillary sinusitis',
'chronic kidney disease stage',
'diabetes mellitus type 2',
'lymph nodes of multiple sites',
'other chronic pain',
'severe abdominal pain',
'squamous cell carcinoma of skin',
'type 2 diabetes mellitus']

snomed_chunks= ['down syndrome', 'adenocarcinoma', 'aortic valve stenosis',
       'atherosclerosis', 'atrial fibrillation',
       'hypertension', 'lung cancer', 'seizure',
       'squamous cell carcinoma', 'stage IIIB', 'mediastinal lymph nodes']

In [ ]:
from IPython.display import display

for chunk in icd_chunks:

    print ('>> ',chunk)
    display(get_codes (icd_lp, chunk, vocab='icd10cm_code'))

>>  obesity


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,obesity,0,6,E669,"[E669, E668, Z6841, Q130, E66, E6601, Z8639, E349, H3550, Z8349, Q5562]","[obesity, abdominal obesity, obese, central obesity, overweight and obesity, morbid obesity, h/o: obesity, severe obesity, centripetal obesity, fh: obesity, truncal obesity]","[0.0000, 0.0218, 0.0264, 0.0278, 0.0290, 0.0355, 0.0403, 0.0414, 0.0449, 0.0473, 0.0482]"


>>  a respiratory tract infection


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,a respiratory tract infection,0,28,J988,"[J988, J069, A499, J22, J209, Z593, T17, J0410, Z1383, J189, P289, J399, J989]","[respiratory tract infection, upper respiratory tract infection, bacterial respiratory infection, acute respiratory infection, bronchial infection, institution-acquired respiratory infection, foreign body in respiratory tract, tracheal infection, suspected respiratory disease, pulmonary infection, respiratory disease, disease of upper respiratory tract, disease of respiratory system]","[0.0248, 0.0507, 0.0675, 0.0737, 0.0751, 0.0786, 0.0834, 0.0868, 0.0906, 0.1005, 0.1033, 0.1024, 0.1031]"


>>  basal cell carcinoma of skin


...

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,severe abdominal pain,0,20,R1084,"[R1084, R101, R109, K388, R1033, R102, F454, K805, R52, M2551, R141, M7918, R1030, R100, M791]","[generalized abdominal pain, upper abdominal pain, intractable abdominal pain, colicky abdominal pain, central abdominal pain, acute abdominal pain, psychosomatic abdominal pain, recurrent abdominal pain, heavy pain, shoulder pain from abdomen, abdominal wind pain, abdominal muscle pain, site of abdominal pain, recurrent acute abdominal pain, sore pain]","[0.0196, 0.0208, 0.0208, 0.0318, 0.0352, 0.0370, 0.0370, 0.0408, 0.0489, 0.0507, 0.0538, 0.0535, 0.0542, 0.0541, 0.0555]"


>>  squamous cell carcinoma of skin


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,squamous cell carcinoma of skin,0,30,C449,"[C449, C443, D049, C4492, C4452, C4499, C4432, C4442, C4A70, C44320, C444, C4491, C799, D04]","[squamous cell carcinoma of skin, squamous cell carcinoma of skin of face, squamous cell carcinoma in situ of skin, scc - squamous cell carcinoma of skin, squamous cell carcinoma of skin of trunk, mucoepidermoid carcinoma of skin, squamous cell carcinoma of skin of and unsp parts of face, squamous cell carcinoma of skin of neck, trabecular cell carcinoma of skin, squamous cell carcinoma of skin of unspecified parts of face, squamous cell carcinoma of scalp, basal cell carcinoma of skin, squamous cell carcinomatosis, carcinoma in situ of skin]","[0.0000, 0.0244, 0.0289, 0.0333, 0.0337, 0.0420, 0.0519, 0.0564, 0.0600, 0.0624, 0.0623, 0.0620, 0.0638, 0.0639]"


>>  type 2 diabetes mellitus


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,type 2 diabetes mellitus,0,23,E11,"[E11, E119, E118, E113, O2411, E139, E1143, E109, E1169, E114, E112, E1151, E1162, E1144, Z863]","[type 2 diabetes mellitus, diabetes mellitus type 2, disorder associated with type 2 diabetes mellitus, disorder of eye with type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus, secondary diabetes mellitus, neurological disorder with type 2 diabetes mellitus, diabetes mellitus, diabetic dyslipidemia associated with type 2 diabetes mellitus, neurological disorder with diabetes type 2, kidney disorder associated with type 2 diabetes mellitus, peripheral circulatory disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus with skin complications, disorder of nervous system due to type 2 diabetes mellitus, history of diabetes mellitus type 2 (situation)]","[0.0000, 0.0134, 0.0212, 0.0357, 0.0360, 0.0397, 0.0455, 0.0462, 0.0499, 0.0523, 0.0537, 0.0548, 0.0549, 0.0546, 0.0554]"


In [ ]:
for chunk in snomed_chunks:

    print ('>> ',chunk)
    display(get_codes (snomed_lp, chunk, vocab='snomed_code', aux_label=True))

>>  down syndrome


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,down syndrome,0,12,41040004,"[41040004, 157019002, 79385002, 408338009, 247545005, 205618003, 254263008, 95429007, 711461002, 25088000, 45056002, 155375008, 266308000, 20741006, 29696001, 711466007, 230675008, 22994000, 255518004, 248199009, 206891002]","[down syndrome, down's syndrome, lowe syndrome, suspected downs syndrome, downy hair, down's syndrome nos, down's syndrome nos, shortened trunk (disorder), decreased lower face height, dimpled structure, abnormal shortened structure, dropsy, dropsy, dropsy, downward displacement, decreased lower anterior face height, slow channel syndrome (disorder), decreased reflex, downward, down facies, [d]dropsy]","[0.0000, 0.0341, 0.0753, 0.0790, 0.0966, 0.1007, 0.1007, 0.0995, 0.1016, 0.1054, 0.1059, 0.1066, 0.1066, 0.1066, 0.1068, 0.1114, 0.1116, 0.1132, 0.1114, 0.1132, 0.1127]","[Complete trisomy 21 syndrome, Down's syndrome, Lowe syndrome, Suspected Downs syndrome, Downy hair, Down's syndrome NOS, Trisomy 21 NOS, Shortened trunk, Decreased lower face height, Dimple, Abnormal shortening, Right heart failure, Dropsy, Hydrops, Prolapse, Decreased lower anterior face height, Slow channel syndrome, Hyporeflexia, Down, Down's facies, [D]Dropsy]","[Condition, Condition, Condition, Observation, Condition, Condition, Condition, Condition, Condition, Observation, Observation, Condition, Condition, Observation, Observation, Condition, Condition, Condition, Observation, Condition, Observation]","[Clinical Finding, Clinical Finding, Clinical Finding, Context-dependent, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Morph Abnormality, Morph Abnormality, Clinical Finding, Clinical Finding, Morph Abnormality, Morph Abnormality, Clinical Finding, Clinical Finding, Clinical Finding, Qualifier Value, Clinical Finding, Context-dependent]"


>>  adenocarcinoma


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,adenocarcinoma,0,13,443961001,"[443961001, 35917007, 403902008, 59367005, 72495009, 58069009, 450948005, 45410002, 423595004, 32968003, 189578007, 134297006, 11671000, 302827006, 86293007, 189614008, 28558000, 89740008]","[adenocarcinoma, adenocarcinoma, adenosquamous carcinoma, adenosquamous carcinoma, mucous adenocarcinoma, ceruminous adenocarcinoma, serrated adenocarcinoma, acinar adenocarcinoma, adenocarcinoma carcinomatosis, inflammatory adenocarcinoma, [m]adenocarcinomas, adenoma and adenocarcinoma, adenocystic carcinoma, adenocystic carcinoma, adenocarcinoid tumor, adenocarcinoid tumor, villous adenocarcinoma, lobular adenocarcinoma]","[0.0000, 0.0000, 0.0401, 0.0401, 0.0432, 0.0496, 0.0513, 0.0520, 0.0546, 0.0563, 0.0586, 0.0593, 0.0594, 0.0594, 0.0615, 0.0615, 0.0601, 0.0617]","[Malignant adenomatous neoplasm, Adenocarcinoma, Adenosquamous cell carcinoma, Adenosquamous carcinoma, Mucinous adenocarcinoma, Ceruminous adenocarcinoma, Serrated adenocarcinoma, Acinar cell carcinoma, Adenocarcinoma carcinomatosis, Inflammatory carcinoma, [M]Adenomas &/or adenocarcinomas, Adenoma and adenocarcinoma, Adenoid cystic carcinoma, Cylindroma, Adenocarcinoid tumor, Adenocarcinoid tumor, Villous adenocarcinoma, Lobular carcinoma]","[Condition, Observation, Condition, Observation, Observation, Observation, Observation, Observation, Condition, Observation, Condition, Observation, Observation, Observation, Observation, Condition, Observation, Observation]","[Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Qualifier Value, Morph Abnormality, Morph Abnormality, Morph Abnormality, Clinical Finding, Morph Abnormality, Morph Abnormality]"


>>  aortic valve stenosis


...

,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,squamous cell carcinoma,0,22,402815007,"[402815007, 28899001, 154605007, 400155002, 425303004, 37304002, 254651007, 89906000, 403904009, 189564006, 189565007, 59529006, 403905005, 403902008, 85956000, 400066006, 252989001]","[squamous cell carcinoma, squamous cell carcinoma, squamous cell carcinoma, squamous cell carcinoma - category, squamous cell carcinomatosis, basisquamous cell carcinoma, cutaneous squamous cell carcinoma, verrucous squamous cell carcinoma, verrucous squamous cell carcinoma, [m]papillomatosis nos, squamous cell carcinoma in situ, squamous cell carcinoma in situ, multiple squamous cell carcinomata, adenosquamous cell carcinoma, adenoid squamous cell carcinoma, intraepithelial squamous cell carcinoma, squamous carcinoma in situ]","[0.0000, 0.0000, 0.0000, 0.0176, 0.0191, 0.0348, 0.0398, 0.0455, 0.0455, 0.0468, 0.0468, 0.0468, 0.0495, 0.0498, 0.0504, 0.0566, 0.0594]","[Squamous cell carcinoma, Squamous cell carcinoma, Squamous cell carcinoma, Squamous cell carcinoma - category, Squamous cell carcinomatosis, Basosquamous carcinoma, Squamous cell carcinoma of skin, Verrucous carcinoma, Verrucous squamous cell carcinoma, [M]Papillomatosis NOS, Squamous cell carcinoma in situ, Squamous cell carcinoma in situ, Multiple squamous cell carcinomata, Adenosquamous cell carcinoma, Adenoid squamous cell carcinoma, Intraepithelial squamous cell carcinoma, Squamous carcinoma in situ]","[Condition, Observation, Condition, Observation, Condition, Observation, Condition, Observation, Condition, Observation, Condition, Observation, Condition, Condition, Observation, Condition, Observation]","[Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality, Clinical Finding, Clinical Finding, Morph Abnormality, Clinical Finding, Morph Abnormality]"


>>  stage IIIB


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,stage IIIB,0,9,261639007,"[261639007, 258226007, 261640009, 258227003, 261638004, 258225006, 261637009, 258223004, 258219007, 258221002, 261615002, 258220001, 261614003, 307204003, 69605008, 52774001, 112240001, 90321006, 39617003, 60333009, 258224005, 70348004, 143013009, 17816005, 307203009]","[stage iiib, stage iiib, stage iiic, stage iiic, stage iiia, stage iiia, stage iic, stage iic, stage ii, stage iib, stage iib, stage iia, stage iia, grade iiib, clinical stage ii e, clinical stage ii a, clinical stage ii d, clinical stage ii s, clinical stage ii c, clinical stage ii, stage iii, gdth iib, ddimer level, clinical stage ii b, grade iiia]","[0.0000, 0.0000, 0.0135, 0.0135, 0.0217, 0.0217, 0.0243, 0.0243, 0.0370, 0.0393, 0.0393, 0.0422, 0.0422, 0.0478, 0.0533, 0.0561, 0.0573, 0.0569, 0.0677, 0.0692, 0.0715, 0.0725, 0.0713, 0.0724, 0.0736]","[Stage 3B, Stage 3b, Stage 3C, Stage IIIc, Stage 3A, Stage IIIa, Stage 2C, Stage 2c, Stage 2, Stage 2b, Stage 2B, Stage IIa, Stage 2A, Grade 3b, Clinical stage II E, Clinical stage II A, Clinical stage II D, Clinical stage II S, Clinical stage II C, Clinical stage II, Stage 3, Pendred's syndrome, DDimer level, Clinical stage II B, Grade 3a]","[Observation, Observation, Observation, Observation, Observation, Observation, Observation, Observation, Observation, Observation, Observation, Observation, Observation, Observation, Condition, Condition, Condition, Condition, Condition, Condition, Observation, Condition, Procedure, Condition, Observation]","[Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Qualifier Value, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Qualifier Value, Clinical Finding, Procedure, Clinical Finding, Qualifier Value]"


>>  mediastinal lymph nodes


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,mediastinal lymph nodes,0,22,245276004,"[245276004, 181760002, 62683002, 62650009, 245275000, 178297005, 260926004, 281779003, 264099006, 364445008, 127931000, 301296002, 127248001, 734421006, 198237001, 20190004, 72372004, 281496003]","[mediastinal lymph nodes, mediastinal lymph node, mediastinal lymph node, stratum germinativum, subclavian lymph node group, mediastinal lymph nodes sampling, mediastinal node status, mediastinal lymphatic structure, mediastinal, mediastinal feature, retrotracheal lymph node (mediastinal), mediastinal observation, neoplasm of mediastinal lymph nodes, specimen from mass of mediastinal lymph node, mediastinal vein, mediastinal vein, pelvic peritoneum structure, structure of pelvic blood vessel]","[0.0000, 0.0226, 0.0226, 0.0342, 0.0458, 0.0511, 0.0635, 0.0670, 0.0782, 0.0777, 0.0813, 0.0849, 0.0879, 0.0892, 0.0882, 0.0882, 0.0875, 0.0961]","[Mediastinal lymph node group, Entire mediastinal lymph node, Mediastinal lymph node structure, Stratum germinativum, Subclavian lymph node group, Mediastinal lymph nodes sampling, Mediastinal node status, Structure of mediastinal lymphatic system, Mediastinal, Mediastinal feature, Retrotracheal lymph node, Mediastinal finding, Neoplasm of mediastinal lymph nodes, Specimen from mass of mediastinal lymph node, Entire mediastinal vein, Structure of mediastinal vein, Pelvic peritoneum structure, Structure of pelvic blood vessel]","[Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site, Procedure, Observation, Spec Anatomic Site, Observation, Observation, Spec Anatomic Site, Condition, Condition, Specimen, Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site, Spec Anatomic Site]","[Body Structure, Body Structure, Body Structure, Body Structure, Body Structure, Procedure, Attribute, Body Structure, Qualifier Value, Observable Entity, Body Structure, Clinical Finding, Clinical Finding, Specimen, Body Structure, Body Structure, Body Structure, Body Structure]"


In [ ]:
clinical_chunks = ['bladder cancer',
 'anemia in chronic kidney disease',
 'castleman disease',
 'congestive heart failure',
 'diabetes mellitus type 2',
 'lymph nodes of multiple sites',
 'malignant melanoma of skin',
 'malignant neoplasm of lower lobe, bronchus',
 'metastatic lung cancer',
 'secondary malignant neoplasm of bone',
 'type 2 diabetes mellitus',
 'type 2 diabetes mellitus/insulin',
 'unsp malignant neoplasm of lymph node']


for chunk in clinical_chunks:

    print ('>> ',chunk)
    
    print ('icd10cm_code')
    display(get_codes (hcc_lp, chunk, vocab='icd10cm_hcc_code', hcc=True))
    
    print ('snomed_code')
    display(get_codes (snomed_lp, chunk, vocab='snomed_code', aux_label=True))

>>  bladder cancer
icd10cm_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,bladder cancer,0,13,C679,"[C679, Z126, D090, D494, C7911]","[bladder cancer, suspected bladder cancer, cancer in situ of urinary bladder, tumor of bladder neck, malignant tumour of bladder neck]","[0.0000, 0.0904, 0.0978, 0.1080, 0.1281]","[1, 1, 1, 1, 1]","[1, 0, 0, 0, 1]","[11, 0, 0, 0, 8]"


snomed_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,bladder cancer,0,13,399326009,"[399326009, 363455001, 425066001, 255108000, 269607003, 154540000, 425231005, 255110003, 139850000, 162582000, 315269000, 393562002, 255109008, 255111004, 92546004, 254932004]","[bladder cancer, bladder cancer, invasive bladder cancer, carcinoma of bladder, carcinoma of bladder, carcinoma of bladder, superficial bladder cancer, adenocarcinoma of bladder, suspected bladder cancer, suspected bladder cancer, suspected bladder cancer, transitional cell carcinoma of bladder, transitional cell carcinoma of bladder, squamous cell carcinoma of bladder, cancer in situ of urinary bladder, tumor of bladder neck]","[0.0000, 0.0000, 0.0539, 0.0666, 0.0666, 0.0666, 0.0809, 0.0881, 0.0904, 0.0904, 0.0904, 0.0880, 0.0880, 0.0913, 0.0978, 0.1080]","[Malignant tumor of urinary bladder, NGB - New growth of bladder, Carcinoma of urinary bladder, invasive, Carcinoma of bladder, Bladder Ca, Carcinoma bladder, Carcinoma of urinary bladder, superficial, Adenocarcinoma of bladder, Suspected bladder cancer, Suspected bladder cancer, Suspected bladder cancer, Transitional cell carcinoma of bladder, Transitional cell carcinoma of bladder, Squamous cell carcinoma of bladder, Cancer in situ of urinary bladder, Tumor of bladder neck]","[Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Observation, Observation, Observation, Condition, Condition, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Context-dependent, Context-dependent, Context-dependent, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]"


>>  anemia in chronic kidney disease
icd10cm_code


...

,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,type 2 diabetes mellitus/insulin,0,31,237599002,"[237599002, 366909003, 44054006, 368051000119109, 422014003, 422099009, 199230006, 137931000119102, 190384004, 421326000]","[insulin treated type 2 diabetes mellitus, insulin treated type 2 diabetes mellitus, type 2 diabetes mellitus, hyperglycemia due to type 2 diabetes mellitus (disorder), disorder associated with type 2 diabetes mellitus, disorder of eye with type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus (disorder), hyperlipidaemia due to type 2 diabetes mellitus, type ii diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus]","[0.0513, 0.0513, 0.0554, 0.0593, 0.0606, 0.0658, 0.0686, 0.0690, 0.0708, 0.0718]","[Insulin treated type 2 diabetes mellitus, Insulin treated Type 2 diabetes mellitus, Type 2 diabetes mellitus, Hyperglycemia due to type 2 diabetes mellitus, Disorder due to type 2 diabetes mellitus, Disorder of eye due to type 2 diabetes mellitus, Pre-existing type 2 diabetes mellitus, Hyperlipidemia due to type 2 diabetes mellitus, Type II diabetes mellitus, Disorder of nervous system due to type 2 diabetes mellitus]","[Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding]"


>>  unsp malignant neoplasm of lymph node
icd10cm_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,billable,hcc_status,hcc_score
0,unsp malignant neoplasm of lymph node,0,36,C779,"[C779, C77, D360, C772, C775, D492, D219]","[secondary and unsp malignant neoplasm of lymph node, unsp, secondary and unspecified malignant neoplasm of lymph nodes, benign neoplasm of lymph node, secondary and unsp malignant neoplasm of intra-abd nodes, secondary and unsp malignant neoplasm of intrapelv nodes, neoplasm of lymph node, benign neoplasm of lymph vessels]","[0.0606, 0.0681, 0.0754, 0.0920, 0.1034, 0.0968, 0.1049]","[1, 0, 1, 1, 1, 1, 1]","[1, 0, 0, 1, 1, 0, 0]","[10, 0, 0, 8, 8, 0, 0]"


snomed_code


,chunks,begin,end,code,all_codes,resolutions,all_distances,gt,concept,aux
0,unsp malignant neoplasm of lymph node,0,36,188379006,"[188379006, 271526003, 188432007, 92197001, 189203003, 188706002, 92202008, 92156008, 188707006, 188713002, 92090002, 92203003, 188392009, 188393004, 178838007, 188709009, 188389005, 188715009, 173840000, 92199003, 92213006, 127232002]","[secondary and unspecified malignant neoplasm of lymph nodes, secondary and unspecified malignant neoplasm of lymph nodes, secondary and unspecified malignant neoplasm of lymph nodes nos, benign neoplasm of lymph node, benign neoplasm of lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of unspecified site, benign neoplasm of lymph nodes of neck, benign neoplasm of intrathoracic lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of head, face and neck, unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrapelvic lymph nodes, benign neoplasm of epitrochlear lymph nodes, benign neoplasm of lymph nodes of upper limb, secondary and unspecified malignant neoplasm of lymph nodes of head, face and neck nos, secondary and unspecified malignant neoplasm of intrathoracic lymph nodes, other specified operations on skull and/or spine and/or bone and/or joint, unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrathoracic lymph nodes, secondary and unspecified malignant neoplasm of submental lymph nodes, unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of multiple sites, other operation on stomach nos, benign neoplasm of lymph nodes of head (disorder), benign neoplasm of mediastinal lymph nodes, neoplasm of lymph node]","[0.0681, 0.0681, 0.0739, 0.0754, 0.0758, 0.0841, 0.0860, 0.0896, 0.0948, 0.0963, 0.0954, 0.0938, 0.0969, 0.0980, 0.0995, 0.0995, 0.0994, 0.1006, 0.1006, 0.0971, 0.0985, 0.0968]","[Secondary and unspecified malignant neoplasm of lymph nodes, Secondary and unspecified malignant neoplasm of lymph nodes, Secondary and unspecified malignant neoplasm of lymph nodes NOS, Benign neoplasm of lymph node, Benign neoplasm of lymph nodes, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of unspecified site, Benign neoplasm of lymph nodes of neck, Benign neoplasm of intrathoracic lymph nodes, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of head, face and neck, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrapelvic lymph nodes, Benign neoplasm of epitrochlear lymph nodes, Benign neoplasm of lymph nodes of upper limb, Secondary and unspecified malignant neoplasm of lymph nodes of head, face and neck NOS, Secondary and unspecified malignant neoplasm of intrathoracic lymph nodes, Other specified operations on skull and/or spine and/or bone and/or joint, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of intrathoracic lymph nodes, Secondary and unspecified malignant neoplasm of submental lymph nodes, Unspecified malignant neoplasm of lymphoid and histiocytic tissue of lymph nodes of multiple sites, Other operation on stomach NOS, Benign neoplasm of lymph nodes of head, Benign neoplasm of mediastinal lymph nodes, Neoplasm of lymph node]","[Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Condition, Procedure, Condition, Condition, Condition, Procedure, Condition, Condition, Condition]","[Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Clinical Finding, Procedure, Clinical Finding, Clinical Finding, Clinical Finding, Procedure, Clinical Finding, Clinical Finding, Clinical Finding]"


##  How to integrate resolvers with NER models in the same pipeline


In [ ]:
documentAssembler = nlp.DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

sentenceDetector = nlp.SentenceDetectorDLModel.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence")

tokenizer = nlp.Tokenizer()\
      .setInputCols(["sentence"])\
      .setOutputCol("token")\

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["sentence", "token"])\
      .setOutputCol("embeddings")

clinical_ner = medical.NerModel.pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = medical.NerConverterInternal() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")\
      .setWhiteList(['PROBLEM'])

c2doc = nlp.Chunk2Doc()\
      .setInputCols("ner_chunk")\
      .setOutputCol("ner_chunk_doc") 

sbert_embedder = nlp.BertSentenceEmbeddings\
      .pretrained("sbiobert_base_cased_mli",'en','clinical/models')\
      .setInputCols(["ner_chunk_doc"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

icd10_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")
  
sbert_resolver_pipeline = nlp.Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        c2doc,
        sbert_embedder,
        icd10_resolver])

data_ner = spark.createDataFrame([[""]]).toDF("text")

sbert_models = sbert_resolver_pipeline.fit(data_ner)

In [ ]:
clinical_note = 'A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting. Two weeks prior to presentation, she was treated with a five-day course of amoxicillin for a respiratory tract infection. She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation. Physical examination on presentation was significant for dry oral mucosa; significantly, her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl, bicarbonate 18 mmol/l, anion gap 20, creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, glycated hemoglobin (HbA1c) 10%, and venous pH 7.27. Serum lipase was normal at 43 U/L. Serum acetone levels could not be assessed as blood samples kept hemolyzing due to significant lipemia. The patient was initially admitted for starvation ketosis, as she reported poor oral intake for three days prior to admission. However, serum chemistry obtained six hours after presentation revealed her glucose was 186 mg/dL, the anion gap was still elevated at 21, serum bicarbonate was 16 mmol/L, triglyceride level peaked at 2050 mg/dL, and lipase was 52 U/L. The β-hydroxybutyrate level was obtained and found to be elevated at 5.29 mmol/L - the original sample was centrifuged and the chylomicron layer removed prior to analysis due to interference from turbidity caused by lipemia again. The patient was treated with an insulin drip for euDKA and HTG with a reduction in the anion gap to 13 and triglycerides to 1400 mg/dL, within 24 hours. Her euDKA was thought to be precipitated by her respiratory tract infection in the setting of SGLT2 inhibitor use. The patient was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely. She had close follow-up with endocrinology post discharge.'

print (clinical_note)

clinical_note_df = spark.createDataFrame([[clinical_note]]).toDF("text")


In [ ]:
icd10_result = sbert_models.transform(clinical_note_df)

In [ ]:
import pyspark.sql.functions as F

pd.set_option('display.max_colwidth', 0)

def get_icd_codes(icd10_res):
    
    icd10_df = icd10_res.select(F.explode(F.arrays_zip(icd10_res.ner_chunk.result, 
                                                       icd10_res.ner_chunk.metadata, 
                                                       icd10_res.icd10cm_code.result, 
                                                       icd10_res.icd10cm_code.metadata)).alias("cols")) \
                                 .select(F.expr("cols['1']['sentence']").alias("sent_id"),
                                         F.expr("cols['0']").alias("ner_chunk"),
                                         F.expr("cols['1']['entity']").alias("entity"), 
                                         F.expr("cols['2']").alias("icd10_code"),
                                         F.expr("cols['3']['all_k_results']").alias("all_codes"),
                                         F.expr("cols['3']['all_k_resolutions']").alias("resolutions")).toPandas()


    
    codes = []
    resolutions = []

    for code, resolution in zip(icd10_df['all_codes'], icd10_df['resolutions']):

        codes.append(code.split(':::'))
        resolutions.append(resolution.split(':::'))

    icd10_df['all_codes'] = codes  
    icd10_df['resolutions'] = resolutions
    
    return icd10_df

In [ ]:
%%time

res_pd = get_icd_codes(icd10_result)

In [ ]:
res_pd.head(15)

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions
0,0,gestational diabetes mellitus,PROBLEM,O2441,"[O2441, O2443, Z8632, Z875, O2431, O2411, O244, O241, O2481]","[gestational diabetes mellitus, postpartum gestational diabetes mellitus, history of gestational diabetes mellitus, history of gestational diabetes mellitus (situation), pre-existing diabetes mellitus in pregnancy, pre-existing diabetes mellitus in pregnancy (disorder), postpartum gestational diabetes mellitus (disorder), pre-existing type 2 diabetes mellitus in pregnancy, other pre-existing diabetes mellitus in pregnancy]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O2411,"[O2411, E118, E11, E139, E119, E113, E1144, Z863, Z8639, E1132, E1143, E115]","[pre-existing type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus, secondary diabetes mellitus, diabetes mellitus type 2, disorder of eye due to type 2 diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus, history of diabetes mellitus type 2 (situation), history of diabetes mellitus type 2, secondary endocrine diabetes mellitus, neurological disorder with type 2 diabetes mellitus, peripheral circulatory disorder due to type 2 diabetes mellitus]"
2,0,T2DM,PROBLEM,P702,"[P702, C801, G121, Q850, C779, C509, C439, E723, C5700, Z7281, C7949, E750, C768, T808]","[tndm2, t2 category, sma2, nf2, pn2 category, pn2a category, pn2c category, ga ii, pt2c category (finding), truancy, pt2c category, tsd, pn2c category (finding), trali]"
3,0,HTG-induced pancreatitis,PROBLEM,K8520,"[K8520, K859, K853, K8590, K85, F102, K858, K8591, K852, K861, K918, K8580, K8592]","[alcohol-induced pancreatitis, pancreatitis, drug induced acute pancreatitis, ap - acute pancreatitis, acute pancreatitis, alcohol-induced chronic pancreatitis, autoimmune pancreatitis, necrotizing pancreatitis, alcohol induced acute pancreatitis, interstitial pancreatitis, post-ercp acute pancreatitis, viral acute pancreatitis, infectious pancreatitis (disorder)]"
4,0,an acute hepatitis,PROBLEM,B15,"[B15, K720, B179, B172, Z0389, B159, K752, K712, B199, B16, K701, B169]","[acute hepatitis a, acute hepatitis, acute infectious hepatitis, acute hepatitis e, acute infectious hepatitis suspected, acute type a viral hepatitis, acute focal hepatitis, toxic liver disease with acute hepatitis, fulminant hepatitis, acute hepatitis b, acute alcoholic hepatitis, acute fulminating viral hepatitis]"
5,0,obesity,PROBLEM,E669,"[E669, E668, Z6841, Q130, E66, E6601, Z8639, E349, H3550, Z8349, Q5562]","[obesity, abdominal obesity, obese, central obesity, overweight and obesity, morbid obesity, h/o: obesity, severe obesity, centripetal obesity, fh: obesity, truncal obesity]"
6,0,a body mass index,PROBLEM,Z6841,"[Z6841, E669, R229, Z681, R223, R221, Z68, R222, R220, R4189, M6258, P299, R1900, R898, M2170, R190, M2180]","[finding of body mass index, observation of body mass index, mass of body region, finding of body mass index (finding), mass of upper limb, head and neck mass, body mass index [bmi], mass of trunk, mass of head, preoccupation with body size, finding of size of upper limb, mass of cardiovascular structure, mass of abdominal cavity structure, increase in circumference, finding of proportion of upper limb, visible abdominal mass, observation of size of upper limb]"
7,0,polyuria,PROBLEM,R35,"[R35, R3581, R358, E232, R31, R350, R8299, N401, E723, O048, R300, R319, E7201, R809, E888, R808, N030, P960, R80, N026]","[polyuria, nocturnal polyuria, polyuric state, polyuric state (disorder), hematuria, micturition frequency and polyuria, uricosuria, pollakiuria, saccharopinuria, oligouria, dysuria, hematuria syndrome, cystinuria, cyclic proteinuria, sialuria, paroxysmal proteinuria, renal hematuria (disorder), oliguria and anuria, proteinuria, persistent hematuria (disorder)]"
8,0,polydipsia,PROBLEM,R631,"[R631, F6389, E232, F639, O40, G475, M7989, R632, R061, H538, I442, G479, G478, G471, Q308, I459, 

Lets apply some HTML formating by using `sparknlp_display` library to see the results of the pipeline in a nicer layout:

In [ ]:
# with light pipeline
light_model = nlp.LightPipeline(sbert_models)

vis = nlp.viz.EntityResolverVisualizer()

# Change color of an entity label
vis.set_label_colors({'PROBLEM':'#008080'})

light_data_icd = light_model.fullAnnotate(clinical_note)

vis.display(light_data_icd[0], 'ner_chunk', 'icd10cm_code')


## BertSentenceChunkEmbeddings

This annotator let users to aggregate sentence embeddings and ner chunk embeddings to get more specific and accurate resolution codes. It works by averaging context and chunk embeddings to get contextual information. Input to this annotator is the context (sentence) and ner chunks, while the output is embedding for each chunk that can be fed to the resolver model. The `setChunkWeight` parameter can be used to control the influence of surrounding context. 

For more information and examples of `BertSentenceChunkEmbeddings` annotator, you can check here: 
[24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/24.1.Improved_Entity_Resolution_with_SentenceChunkEmbeddings.ipynb)

### ICD10CM with BertSentenceChunkEmbeddings

Lets do the same process by using `BertSentenceEmbeddings` annotator and compare the results. We will create a new pipeline by using this annotator with SentenceEntityResolverModel.

In [ ]:
#Get average sentence-chunk Bert embeddings
sentence_chunk_embeddings = medical.BertSentenceChunkEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence", "ner_chunk"])\
    .setOutputCol("sbert_embeddings")\
    .setCaseSensitive(False)\
    .setChunkWeight(0.5) #default : 0.5


icd10_resolver = medical.SentenceEntityResolverModel.pretrained("sbiobertresolve_icd10cm_augmented","en", "clinical/models") \
      .setInputCols(["sbert_embeddings"]) \
      .setOutputCol("icd10cm_code")\
      .setDistanceFunction("EUCLIDEAN")
  
resolver_pipeline_SCE = nlp.Pipeline(
    stages = [
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter,
        sentence_chunk_embeddings,
        icd10_resolver])


empty_data = spark.createDataFrame([['']]).toDF("text")
model_SCE = resolver_pipeline_SCE.fit(empty_data)

In [ ]:
icd10_result_SCE = model_SCE.transform(clinical_note_df)

In [ ]:
%%time

res_SCE_pd = get_icd_codes(icd10_result_SCE)

CPU times: user 646 ms, sys: 130 ms, total: 776 ms
Wall time: 2min 8s


In [ ]:
res_SCE_pd.head(15)

,sent_id,ner_chunk,entity,icd10_code,all_codes,resolutions
0,0,gestational diabetes mellitus,PROBLEM,O244,"[O244, O2443, O2441, O24434, O2442, O24414, O2413]","[gestational diabetes mellitus, class d, gestational diabetes mellitus in the puerperium, gestational diabetes mellitus, class a>1<, gestational diabetes mellitus in the puerperium, insulin controlled, gestational diabetes mellitus in childbirth, gestational diabetes mellitus in pregnancy, insulin controlled, pre-existing type 2 diabetes mellitus, in the puerperium]"
1,0,subsequent type two diabetes mellitus,PROBLEM,O2413,"[O2413, E116, E1143, O2411, E1110, E1133, E114, O241, O2412, E111, K318, E1169, E113, O244, E112, E109, E110]","[pre-existing type 2 diabetes mellitus, in the puerperium, gastroparesis co-occurrent and due to type 2 diabetes mellitus, gastroparesis with type 2 diabetes mellitus, pregnancy and type 2 diabetes mellitus, ketoacidosis due to type 2 diabetes mellitus, moderate nonproliferative retinopathy co-occurrent and due to type 2 diabetes mellitus, cranial nerve palsy co-occurrent and due to type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus in pregnancy, pre-existing type 2 diabetes mellitus, in childbirth, ketoacidosis due to type 2 diabetes mellitus (disorder), diabetic gastroparesis associated with type 2 diabetes mellitus, angina due to type 2 diabetes mellitus, rubeosis iridis co-occurrent and due to type 2 diabetes mellitus, gestational diabetes mellitus, class d, persistent microalbuminuria due to type 2 diabetes mellitus, acanthosis nigricans due to type 2 diabetes mellitus, ketoacidotic coma due to type 2 diabetes mellitus]"
2,0,T2DM,PROBLEM,R701,"[R701, S3202, E161, E1137, P702, T452X, S2202, L658, K912, Z379, E891, I8702, O4291, I8709, E728]","[reticulcytosis after b12, stable burst fx second lum vert, subs for fx w delay heal, ectopic igf-2 hypoglycemia (disorder), type 2 diab with diab mclr edema, resolved fol trtmt, l eye, diabetes mellitus, transient neonatal 2 (disorder), reticulcytosis after b12 (finding), stable burst fx second thor vert, subs for fx w delay heal, postinfectional hypotrichosis, postprocedural steatorrhea, prolonged second stage - delivered (disorder), post-surgical hypoinsulinemia (disorder), postthrom syndrome w inflammation of bilateral low extrm, pretrm prem rom, unsp time betw rupt and onset labr, 2nd tri, postthrom syndrome w oth comp of bilateral low extrm, d,l-2-hydroxyglutaric acidemia]"
3,0,HTG-induced pancreatitis,PROBLEM,E139,"[E139, E1069, K8532, K918, K903, K8502, K8522, K8512, K8510, K8580, K859, K8590, K8591, K852, F102, K853, K8531]","[congenital insulin-dependent diabetes mellitus with fatal secretory diarrhea, congenital insulin-dependent diabetes mellitus with fatal secretory diarrhoea, drug induced acute pancreatitis with infected necrosis, post-ercp acute pancreatitis (disorder), postprocedural pancreatic steatorrhea (disorder), idiopathic acute pancreatitis with infected necrosis, alcohol induced acute pancreatitis with infected necrosis, biliary acute pancreatitis with infected necrosis, gallstone acute pancreatitis (disorder), viral acute pancreatitis (disorder), acute recurrent pancreatitis (disorder), acute hemorrhagic pancreatitis (disorder), acute necrotizing pancreatitis (disorder), alcohol-induced acute pancreatitis (disorder), alcohol-induced chronic pancreatitis, drug-induced acute pancreatitis (disorder), drug induced acute pancreatitis with uninfected necrosis]"
4,0,an acute hepatitis,PROBLEM,B169,"[B169, B159, B162, B179, B160, K712, E8021, F1098, B181, O266, O9843, B150]","[acute fulminating type b viral hepatitis (disorder), acute fulminating type a viral hepatitis (disorder), hepatic coma due to acute hepatitis b, acute fulminating viral hepatitis (disorder), acute viral hepatitis b with hepatic coma, toxic liver disease with acute hepatitis (disorder), acute intermittent (hepatic) porphyria, ketoacidosis due to acute alcoholic intoxication, chronic viral hepatitis

In [ ]:
icd10_SCE_lp = nlp.LightPipeline(model_SCE)

light_result = icd10_SCE_lp.fullAnnotate(clinical_note)

visualiser = nlp.viz.EntityResolverVisualizer()

# Change color of an entity label
visualiser.set_label_colors({'PROBLEM':'#008080'})

visualiser.display(light_result[0], 'ner_chunk', 'icd10cm_code')

**Lets compare the results that we got from these two methods.**

In [ ]:
sentence_df = icd10_result.select(F.explode(F.arrays_zip(icd10_result.sentence.metadata, 
                                                         icd10_result.sentence.result)).alias("cols")) \
                                .select( F.expr("cols['0']['sentence']").alias("sent_id"),
                                         F.expr("cols['1']").alias("sentence_all")).toPandas()

comparison_df = pd.merge(res_pd.loc[:,'sent_id':'resolutions'],res_SCE_pd.loc[:,'sent_id':'resolutions'], on=['sent_id',"ner_chunk", "entity"], how='inner')
comparison_df.columns=['sent_id','ner_chunk', 'entity', 'icd10_code', 'all_codes', 'resolutions', 'icd10_code_SCE', 'all_codes_SCE', 'resolutions_SCE']

comparison_df = pd.merge(sentence_df, comparison_df,on="sent_id").drop('sent_id', axis=1)
comparison_df.head(15)

,sentence_all,ner_chunk,entity,icd10_code,all_codes,resolutions,icd10_code_SCE,all_codes_SCE,resolutions_SCE
0,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",gestational diabetes mellitus,PROBLEM,O2441,"[O2441, O2443, Z8632, Z875, O2431, O2411, O244, O241, O2481]","[gestational diabetes mellitus, postpartum gestational diabetes mellitus, history of gestational diabetes mellitus, history of gestational diabetes mellitus (situation), pre-existing diabetes mellitus in pregnancy, pre-existing diabetes mellitus in pregnancy (disorder), postpartum gestational diabetes mellitus (disorder), pre-existing type 2 diabetes mellitus in pregnancy, other pre-existing diabetes mellitus in pregnancy]",O244,"[O244, O2443, O2441, O24434, O2442, O24414, O2413]","[gestational diabetes mellitus, class d, gestational diabetes mellitus in the puerperium, gestational diabetes mellitus, class a>1<, gestational diabetes mellitus in the puerperium, insulin controlled, gestational diabetes mellitus in childbirth, gestational diabetes mellitus in pregnancy, insulin controlled, pre-existing type 2 diabetes mellitus, in the puerperium]"
1,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, presented with a one-week history of polyuria, polydipsia, poor appetite, and vomiting.",subsequent type two diabetes mellitus,PROBLEM,O2411,"[O2411, E118, E11, E139, E119, E113, E1144, Z863, Z8639, E1132, E1143, E115]","[pre-existing type 2 diabetes mellitus, disorder associated with type 2 diabetes mellitus, type 2 diabetes mellitus, secondary diabetes mellitus, diabetes mellitus type 2, disorder of eye due to type 2 diabetes mellitus, disorder of nervous system due to type 2 diabetes mellitus, history of diabetes mellitus type 2 (situation), history of diabetes mellitus type 2, secondary endocrine diabetes mellitus, neurological disorder with type 2 diabetes mellitus, peripheral circulatory disorder due to type 2 diabetes mellitus]",O2413,"[O2413, E116, E1143, O2411, E1110, E1133, E114, O241, O2412, E111, K318, E1169, E113, O244, E112, E109, E110]","[pre-existing type 2 diabetes mellitus, in the puerperium, gastroparesis co-occurrent and due to type 2 diabetes mellitus, gastroparesis with type 2 diabetes mellitus, pregnancy and type 2 diabetes mellitus, ketoacidosis due to type 2 diabetes mellitus, moderate nonproliferative retinopathy co-occurrent and due to type 2 diabetes mellitus, cranial nerve palsy co-occurrent and due to type 2 diabetes mellitus, pre-existing type 2 diabetes mellitus in pregnancy, pre-existing type 2 diabetes mellitus, in childbirth, ketoacidosis due to type 2 diabetes mellitus (disorder), diabetic gastroparesis associated with type 2 diabetes mellitus, angina due to type 2 diabetes mellitus, rubeosis iridis co-occurrent and due to type 2 diabetes mellitus, gestational diabetes mellitus, class d, persistent microalbuminuria due to type 2 diabetes mellitus, acanthosis nigricans due to type 2 diabetes mellitus, ketoacidotic coma due to type 2 diabetes mellitus]"
2,"A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, associated with an acute hepatitis, and obesity with a body mass index (BMI) of 33.5 kg/m2, pr